In [1]:
import pickle
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# Import aggregation module
import imp
aggregation = imp.load_source('aggregation', '../../aggregation/aggregation.py')

In [2]:
# Load data
with open('../indiv.pickle', 'rb') as file:
    indiv = pickle.load(file)
with open('../hhold.pickle', 'rb') as file:
    hhold = pickle.load(file)

## Transform indiv Using Best Aggregation Method
The best transformation is determined by Spearman correlation for numeric variables and Chi-squared for categorical.

In [3]:
# Set id, iid as multi-index for easier aggregation
X = indiv.set_index(['id', 'iid']).drop(labels=['poor'], axis=1)
# Set id as index and keep only the target variable
Y = pd.DataFrame(hhold.set_index(['id'])['poor'])

In [4]:
# Choose the best aggregation functions based on spearman correlation (for numeric)
# and chi-squared test (for categorical) data. Fit and transform.
corr_agg = aggregation.Aggregate(X, Y=Y, groupby='id')
result_agg = corr_agg.fit_transform()
transform_dict = corr_agg.col_to_func

## For hhold, Convert Categorical to Dummy Variables

In [5]:
def turn_to_dummies (df, protected_cols):
    '''A function that finds and converts categorical data to dummy variables.
    @ Parameters:
    --------------
    df: the original dataframe
    protected_cols: a list of features that should be preserved as is (indices, target, etc.)
    '''
    # Get various lists of features for easy pandas slicing
    num_feat_list = [col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col not in protected_cols]
    cat_feat_list = [col for col in df.columns if df[col].dtype == 'object' and col not in protected_cols]
    # Dataframe with all categorical features converted into dummies
    df_dummies = pd.get_dummies(df[cat_feat_list])
    # Merge the protected and numeric columns with the dummy categorical data
    return pd.merge(df[protected_cols + num_feat_list], df_dummies, left_index=True, right_index=True)

In [6]:
# Convert cats to dummies
hhold = turn_to_dummies(hhold, ['id', 'poor'])

## Merge hhold and indiv

In [8]:
# Set 'id' as index of household
hhold = hhold.set_index(['id'])

In [9]:
# Merge the household and individal data
agg_df = pd.merge(hhold, result_agg, left_index=True, right_index=True)

In [10]:
agg_df.head()

,poor,h_num_001,h_num_002,h_num_003,h_num_004,h_cat_001_DkQlr,h_cat_001_JhtDR,h_cat_002_GUusz,h_cat_002_alLXR,h_cat_003_BIZns,...,i_cat_037_FUUXv,i_cat_037_GtHel,i_cat_037_juMSt,i_cat_038_ALcKg,i_cat_038_JTCKs,i_cat_038_UaIsy,i_cat_038_dSJoN,i_cat_038_vhhVz,i_num_001,i_num_002
id,,,,,,,,,,,,,,,,,,,,,
46107,False,-16.0,21,-2.0,-7,0,1,1,0,0,...,False,True,1,False,1,0,0,False,4.0,211
82739,False,-10.0,12,-3.0,-1,0,1,1,0,0,...,False,True,1,False,0,0,1,False,4.0,116
9646,True,-4.0,12,-3.0,-7,0,1,1,0,1,...,False,True,1,False,1,0,0,False,4.0,136
10975,True,-4.0,3,-4.0,-1,0,1,1,0,0,...,False,True,1,False,1,0,0,False,4.0,126
16463,True,-4.0,21,-4.0,-5,0,1,0,1,0,...,True,True,0,False,1,0,0,False,4.0,136


In [11]:
agg_df.shape

(8203, 1132)

## Scale Numeric Data

In [12]:
# Gather numeric cols
num_cols = [col for col in agg_df if '_num' in col]

In [13]:
# Scale using MinMaxScaler
min_max = MinMaxScaler()
agg_df[num_cols] = min_max.fit_transform(agg_df[num_cols])

In [14]:
agg_df.head()

,poor,h_num_001,h_num_002,h_num_003,h_num_004,h_cat_001_DkQlr,h_cat_001_JhtDR,h_cat_002_GUusz,h_cat_002_alLXR,h_cat_003_BIZns,...,i_cat_037_FUUXv,i_cat_037_GtHel,i_cat_037_juMSt,i_cat_038_ALcKg,i_cat_038_JTCKs,i_cat_038_UaIsy,i_cat_038_dSJoN,i_cat_038_vhhVz,i_num_001,i_num_002
id,,,,,,,,,,,,,,,,,,,,,
46107,False,0.818182,0.621849,0.4,0.705882,0,1,1,0,0,...,False,True,1,False,1,0,0,False,0.0,0.298969
82739,False,0.909091,0.584034,0.2,0.882353,0,1,1,0,0,...,False,True,1,False,0,0,1,False,0.0,0.103093
9646,True,1.000000,0.584034,0.2,0.705882,0,1,1,0,1,...,False,True,1,False,1,0,0,False,0.0,0.144330
10975,True,1.000000,0.546218,0.0,0.882353,0,1,1,0,0,...,False,True,1,False,1,0,0,False,0.0,0.123711
16463,True,1.000000,0.621849,0.0,0.764706,0,1,0,1,0,...,True,True,0,False,1,0,0,False,0.0,0.144330


In [15]:
# Output aggregate data and transformation dictionary as pickle files
with open('agg_df.pickle', 'wb') as f:
    pickle.dump(agg_df, f, pickle.HIGHEST_PROTOCOL)
with open('indiv_transform_dict.pickle', 'wb') as f:
    pickle.dump(transform_dict, f, pickle.HIGHEST_PROTOCOL)